In [2]:
!git clone https://github.com/huggingface/diffusers
!cd diffusers
!pip install

fatal: destination path 'diffusers' already exists and is not an empty directory.
ERROR: You must give at least one requirement to install (see "pip help install")


In [6]:
!pip install -r diffusers/examples/text_to_image/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 128.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 120.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 36.0 MB/s eta 0:00:00


In [14]:
!accelerate config default

Configuration already exists at /home/jupyter/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.


In [17]:
!pip install huggingface-hub


In [18]:
from huggingface_hub import login

# Login to Hugging Face
login()

In [24]:
MODEL_NAME="CompVis/stable-diffusion-v1-4"
dataset_name="colonoscopy_data_for_vqa"
OUTPUT_DIR="new_model"

In [25]:
# The --use_8bit_adam flag is crucial to be able to train on the T4 GPU which has only 15GB of memory
!accelerate launch diffusers/examples/text_to_image/train_text_to_image.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --dataset_name=$dataset_name \
  --use_ema \
  --use_8bit_adam \
  --resolution=512 --center_crop --random_flip \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --gradient_checkpointing \
  --mixed_precision="fp16" \
  --max_train_steps=100 \
  --learning_rate=1e-05 \
  --max_grad_norm=1 \
  --lr_scheduler="constant" --lr_warmup_steps=0 \
  --output_dir="model1"

/opt/conda/envs/pytorch/lib/python3.10/site-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)
/opt/conda/envs/pytorch/lib/python3.10/site-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)
/o

In [30]:
from diffusers import StableDiffusionPipeline
import torch
model_path = "finetuned-stable-diffusion"
pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16)
pipe.to("cuda")

image = pipe(prompt="Generate an image with an an abnormality located in the center, upper-left, upper-right, lower-left, lower-right, center-left, center-right, upper-center and lower-center.").images[0]
image.save("gen-finetuned.png")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
import os
import csv
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image

# Set up the pipeline
model_path = "finetuned-stable-diffusion"
pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16)
pipe.to("cuda")

# Define file paths
prompt_file_path = "ImageCLEFmed-MEDVQA-GI-2024-Testing-Propmpts.txt"
output_folder = "synthetic"
csv_file_path = "prompt_and_data.csv"

# Create output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Open the prompt file and read prompts
with open(prompt_file_path, 'r') as file:
    prompts = file.readlines()

# Open the CSV file for writing
with open(csv_file_path, 'w', newline='') as csvfile:
    fieldnames = ['prompt', 'image_name', 'image_path']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Generate images for each prompt and save them
    for i, prompt in enumerate(prompts):
        prompt = prompt.strip()
        if prompt:  # Check if prompt is not empty
            image = pipe(prompt=prompt).images[0]
            image_name = f"gen-finetuned-{i+1}.png"
            image_path = os.path.join(output_folder, image_name)
            image.save(image_path)

            # Write prompt and image details to the CSV file
            writer.writerow({'prompt': prompt, 'image_name': image_name, 'image_path': image_path})

print("Image generation complete. Prompts and image data saved to CSV.")
